In [1]:
from yikit.models import Objective
from yikit.models import SupportVectorRegressor, GBDTRegressor, LinearModelRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes
from sklearn.neural_network import MLPRegressor
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, train_test_split
import optuna
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
SEED = 334

In [3]:
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.DataFrame(diabetes.target, columns=['PRICE'])
display(X.head(), y.head())

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


,PRICE
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
kf = KFold(n_splits=5)

In [5]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)
estimators = [MLPRegressor(max_iter=1000), GBDTRegressor(), RandomForestRegressor(), SupportVectorRegressor(), GBDTRegressor(), LinearModelRegressor()]
def get_best_estimator(estimator):
    objective = Objective(estimator=estimator, X=X_train, y=y_train.ravel(), scoring='neg_mean_squared_error', random_state=SEED, cv=kf)
    study = optuna.create_study(sampler=objective.sampler, direction='maximize')
    study.optimize(objective, n_trials=10, n_jobs=1)
    best_estimator_ = clone(estimator)
    best_estimator_.set_params(**objective.fixed_params_, **study.best_params)
    return best_estimator_

optuna.logging.disable_default_handler()
[get_best_estimator(estimator) for estimator in tqdm(estimators)]

  0%|          | 0/6 [00:00<?, ?it/s]

/Users/yu9824/miniforge3/envs/yikit/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yu9824/miniforge3/envs/yikit/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yu9824/miniforge3/envs/yikit/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/yu9824/miniforge3/envs/yikit/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
 

In [ ]:
optuna.logging.enable_default_handler()